In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset

In [2]:
# mnist 데이터 다운
train_dataset = dset.MNIST(root=".", train=True, transform=transforms.ToTensor(), download=True)

test_dataset = dset.MNIST(root=".", train=False, transform=transforms.ToTensor())

100%|██████████| 9.91M/9.91M [00:04<00:00, 2.17MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 157kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.41MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.28MB/s]


In [3]:
# 데이터 로더 설정

batch_size = 100

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# RNN 모델 정의
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

        out, hn = self.rnn(x, h0)

        out = self.fc(out[:, -1, :])

        return out

In [5]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim).to(device)

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
num_epochs = 20


for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        images = images.view(-1, input_dim, input_dim)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        images = images.view(-1, 28, input_dim)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum()

    accuracy = 100 * correct / total

    print("Epoch: {}. Loss: {}. Accuracy: {}".format(epoch, loss.item(), accuracy))

Epoch: 0. Loss: 2.3044214248657227. Accuracy: 11.34999942779541
Epoch: 1. Loss: 2.302661657333374. Accuracy: 12.739999771118164
Epoch: 2. Loss: 2.2625679969787598. Accuracy: 18.709999084472656
Epoch: 3. Loss: 1.5503034591674805. Accuracy: 43.0
Epoch: 4. Loss: 1.218593955039978. Accuracy: 58.38999938964844
Epoch: 5. Loss: 0.8290705680847168. Accuracy: 75.0
Epoch: 6. Loss: 0.5029712319374084. Accuracy: 81.40999603271484
Epoch: 7. Loss: 0.4498375952243805. Accuracy: 83.63999938964844
Epoch: 8. Loss: 0.4952700436115265. Accuracy: 86.62999725341797
Epoch: 9. Loss: 0.3397103250026703. Accuracy: 88.04000091552734
Epoch: 10. Loss: 0.30995312333106995. Accuracy: 91.31999969482422
Epoch: 11. Loss: 0.37690165638923645. Accuracy: 91.0
Epoch: 12. Loss: 0.17120212316513062. Accuracy: 93.43000030517578
Epoch: 13. Loss: 0.34022143483161926. Accuracy: 91.80999755859375
Epoch: 14. Loss: 0.2681252360343933. Accuracy: 94.0199966430664
Epoch: 15. Loss: 0.12808431684970856. Accuracy: 93.7699966430664
Epoch: